# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [102]:
# import libraries
import re
import pickle
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///message_categories.db')
df = pd.read_sql('SELECT * FROM message_categories', engine)
X = df.loc[:, 'message']
Y = df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [3]:
X.iloc[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [4]:
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    '''Tokenize textual data to be processed by TFIDF vectorizers
    
        params:
            text - a string of textual data
        returns:
            clean_tokens - tokens from text
    '''
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    clean_tokens = list()
    tokens = word_tokenize(text)
    
    removed_stopwords = [token for token in tokens if token not in stopwords.words('english')]
    
    clean_tokens = [lemmatizer.lemmatize(word, pos='v').lower().strip() for word in removed_stopwords]
    clean_tokens = [lemmatizer.lemmatize(token, pos='n').lower().strip() for token in clean_tokens]
    return clean_tokens

In [5]:
for message in X.iloc[0:30]:
    print(tokenize(message))

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
['is', 'hurricane']
['looking', 'someone', 'name']
['un', 'report', 'leogane', '80', '90', 'destroy', 'only', 'hospital', 'st', 'croix', 'function', 'need', 'supply', 'desperately']
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']
['information', 'national', 'palace']
['storm', 'sacred', 'heart', 'jesus']
['please', 'need', 'tent', 'water', 'we', 'silo', 'thank']
['i', 'would', 'like', 'receive', 'message', 'thank']
['i', 'croix', 'de', 'bouquet', 'we', 'health', 'issue', 'they', 'worker', 'santo', '15', 'area', 'croix', 'de', 'bouquet']
['there', 'nothing', 'eat', 'water', 'starve', 'thirsty']
['i', 'petionville', 'i', 'need', 'information', 'regard', '4636']
['i', 'thomassin', 'number', '32', 'area', 'name', 'pyron', 'i', 'would', 'like', 'water', 'thank', 'god', 'fine', 'desperately', 'need', 'water', 'thanks']
['let', 'together', 'need', 'food', 'delma', '75', 'didine', 'area']
['more', '

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf_trans', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=5)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [9]:
(y_pred == y_test).mean().mean()

0.94300342454141672

In [10]:
for i, class_ in enumerate(y_test.columns):
    print(class_, classification_report(y_test.loc[:, class_].values, y_pred[:, i]))

related              precision    recall  f1-score   support

          0       0.62      0.45      0.52      1540
          1       0.84      0.91      0.87      4961
          2       0.27      0.23      0.25        53

avg / total       0.78      0.80      0.79      6554

request              precision    recall  f1-score   support

          0       0.90      0.96      0.93      5420
          1       0.72      0.50      0.59      1134

avg / total       0.87      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.75      0.79      0.77      3811
          1       0.69      0.63      0.66      2743

avg / total       0.72      0.72      0.72      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd0ed26bf28>, vocabulary=None)),
  ('tfidf_trans',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
               oob_score=False, random_state=

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    'clf__estimator__min_samples_split':[2, 5]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, n_jobs=-1)

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
y_pred_cv = cv.predict(X_test)
(y_pred_cv == y_test).mean().mean()

0.94400366188587137

In [19]:
cv.best_params_

{'clf__estimator__min_samples_split': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
for i, class_ in enumerate(y_test.columns):
    print(class_, classification_report(y_test.loc[:, class_].values, y_pred_cv[:, i]))

related              precision    recall  f1-score   support

          0       0.62      0.41      0.49      1540
          1       0.83      0.92      0.87      4961
          2       0.58      0.13      0.22        53

avg / total       0.78      0.80      0.78      6554

request              precision    recall  f1-score   support

          0       0.90      0.96      0.93      5420
          1       0.73      0.50      0.60      1134

avg / total       0.87      0.88      0.87      6554

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related              precision    recall  f1-score   support

          0       0.77      0.80      0.78      3811
          1       0.70      0.66      0.68      2743

avg / total       0.74      0.74      0.74      6554

medical_help              precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
for sentence in nltk.sent_tokenize(X_train.iloc[0:]):
    print(sentence)
    print(nltk.pos_tag(tokenize(sentence)))

In [69]:
test = nltk.pos_tag(tokenize(X_train.iloc[29]))

In [98]:
def model_pipeline():
    '''Build model pipelne
    
       returns:
           pipeline -  a Pipeline object that can be fit to the data
    '''
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf_trans', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
    return pipeline

In [99]:
improved_pipeline = model_pipeline()
improved_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [100]:
y_pred_ip = improved_pipeline.predict(X_test)

In [101]:
(y_pred_ip == y_test).mean().mean()

0.93416658190079027

The two models are about the same, I'll go with the random forest. If I had more time, I would have tried to make more features than just the Tfidf. Probably word counts, number of capital letters are a couple of ideas I would have liked to have tried.

### 9. Export your model as a pickle file

In [104]:
filename = 'rand_for_class.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.